In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

import pandas as pd
import numpy as np
import io
import time
import datetime
import csv

import requests
from bs4 import BeautifulSoup

In [8]:
options = webdriver.ChromeOptions()
options.add_argument("--lang=vi")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=options)
driver.set_page_load_timeout(20)

In [9]:
driver.get('https://www.google.com.vn/')

In [8]:
driver.get('https://twitter.com/i/flow/login')
time.sleep(3)

In [9]:
#jakejills88591

email_holder = driver.find_element(by=By.TAG_NAME, value='input')
email_holder.send_keys('nmkhoi.iac@gmail.com')
time.sleep(3)

next_button = driver.find_element(by=By.XPATH, value='//*[@id="layers"]/div/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div[2]/div/div/div/div[6]/div/span')
next_button.click()
time.sleep(3)

username_holder = driver.find_element(by=By.TAG_NAME, value='input')
username_holder.send_keys('jakejills88591')
time.sleep(3)

next_button = driver.find_element(by=By.XPATH, value='//*[@id="layers"]/div/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div[2]/div[2]/div/div/div/div')
next_button.click()
time.sleep(3)

password_holder = driver.find_elements(by=By.TAG_NAME, value='input')[1]
password_holder.send_keys('Storible456')
time.sleep(3)

login_button = driver.find_element(by=By.XPATH, value='//*[@id="layers"]/div/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div[2]/div[2]/div/div[1]/div/div/div/div/span')
login_button.click()
time.sleep(10)

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=114.0.5735.199)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x008FA813+48355]
	(No symbol) [0x0088C4B1]
	(No symbol) [0x00795358]
	(No symbol) [0x0077D293]
	(No symbol) [0x007DE37B]
	(No symbol) [0x007EC473]
	(No symbol) [0x007DA536]
	(No symbol) [0x007B82DC]
	(No symbol) [0x007B93DD]
	GetHandleVerifier [0x00B5AABD+2539405]
	GetHandleVerifier [0x00B9A78F+2800735]
	GetHandleVerifier [0x00B9456C+2775612]
	GetHandleVerifier [0x009851E0+616112]
	(No symbol) [0x00895F8C]
	(No symbol) [0x00892328]
	(No symbol) [0x0089240B]
	(No symbol) [0x00884FF7]
	BaseThreadInitThunk [0x75AD7D59+25]
	RtlInitializeExceptionChain [0x7757B74B+107]
	RtlClearBits [0x7757B6CF+191]


In [26]:
# search_box = [i for i in driver.find_elements(by=By.TAG_NAME, value='input') if i.get_attribute('placeholder') == 'Search Twitter'][0]

In [15]:
query = '#chatgpt'
query = query.replace('#','%23')
query = query.replace('@','%40')
url_query = 'https://twitter.com/search?q={}&src=typed_query'.format(query)
driver.get(url_query)

https://twitter.com/search?q=%23chatgpt&src=typed_query


In [28]:
import re
hashtag_pattern = r'#\S+'
username_pattern = r'@\S+'
url_pattern = r'http.+?(?="|<)'

In [29]:
def scrolling():
    SCROLL_PAUSE_TIME = 0.5

    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")

In [30]:
max_tweet = 10
def scraping():
    final_data = pd.DataFrame(columns=['username','text','date','hashtag','external_links','reply_count','retweet_count','like_count','mention_users'])
    while len(final_data) <= max_tweet:
        articles = driver.find_elements(by=By.TAG_NAME, value='article')
        for article in articles:
            try:
                username = re.findall(username_pattern,[i.text for i in article.find_elements(by=By.TAG_NAME, value='div') if i.get_attribute('data-testid') == 'User-Name'][0])[0]
                text = [i.text for i in article.find_elements(by=By.TAG_NAME, value='div') if i.get_attribute('data-testid') == 'tweetText'][0]
                date = article.find_element(by=By.TAG_NAME, value='time').get_attribute('datetime')
                date = date[:date.find('T')]
                hashtag = re.findall(hashtag_pattern,text)
                external_links = re.findall(url_pattern,article.text)
                reply_count = [i.text for i in article.find_elements(by=By.TAG_NAME, value='div') if i.get_attribute('data-testid') == 'reply'][0]
                retweet_count = [i.text for i in article.find_elements(by=By.TAG_NAME, value='div') if i.get_attribute('data-testid') == 'retweet'][0]
                like_count = [i.text for i in article.find_elements(by=By.TAG_NAME, value='div') if i.get_attribute('data-testid') == 'like'][0]
                mention_users = re.findall(username_pattern,text)

                final_data = pd.concat([final_data, pd.DataFrame.from_records([{
                    'username':username,
                    'text':text,
                    'date':date,
                    'hashtag':hashtag,
                    'external_links':external_links,
                    'reply_count':reply_count,
                    'retweet_count':retweet_count,
                    'like_count':like_count,
                    'mention_users':mention_users
                    }])])
                time.sleep(2)
            except Exception as e:
                continue
        
        final_data.loc[final_data.astype(str).drop_duplicates().index]
        scrolling()
        time.sleep(5)
    # driver.quit()
    return final_data.head(max_tweet)

In [31]:
final_data = scraping()

In [32]:
final_data.dtypes

username          object
text              object
date              object
hashtag           object
external_links    object
reply_count       object
retweet_count     object
like_count        object
mention_users     object
dtype: object

In [33]:
len(final_data)

10

In [34]:
final_data

,username,text,date,hashtag,external_links,reply_count,retweet_count,like_count,mention_users
0,@ed__marquez,We’re verified in the #ChatGPT #Plugins Store ...,2023-06-25,"[#ChatGPT, #Plugins]",[],12,90,246,[@Hedera]
0,@DiptiOjha16,Do you know? now you can even generate power p...,2023-06-19,"[#ChatGPT, #AI, #technology]",[],6,8,34,[]
0,@TheAdityaPatro,Fascinating #AI developed artwork at the MoMA ...,2023-06-23,"[#AI, #OPENAI, #GenerativeAI, #ChatGPT, #DataS...",[],15,94,177,[@refikanadol]
0,@AminaAkka,4936223put same list \n#飞机号 #ChatGPT,2023-06-26,"[#飞机号, #ChatGPT]",[],14,11,11,[]
0,@iampragyansh,In this World of AI advancement\nWhat if we co...,2023-06-24,"[#elonmusk, #chatgpt, #meditation, #ai, #shiva...",[],10,30,224,[]
0,@ant0723,The #ChatGPT#Plugin Store officially has 500 p...,2023-06-21,[#ChatGPT#Plugin],[],5,5,44,[]
0,@AminaAkka,4351710analysis cup crime \n#飞机号 #ChatGPT,2023-06-26,"[#飞机号, #ChatGPT]",[],14,11,11,[]
0,@DrasmaJabeen1,Using #ChatGPT for language editing in scienti...,2023-06-24,"[#ChatGPT, #phd, #AcademicTwitter, #AcademicCh...",[],8,112,403,[]
0,@sonu_monika,#ChatGPT Prompts Cheat Sheet for beginners by ...,2023-06-20,"[#ChatGPT, #ArtificialIntelligence, #automatio...",[],10,40,80,[]
0,@ilda113100,2091009want cut catch \n#ChatGPT账号 #ChatGPT,2023-06-26,"[#ChatGPT账号, #ChatGPT]",[],12,10,10,[]


In [1]:
text = '@abcd'
if '@' in text:
    print('yes')

yes


In [3]:
if text[0] == '@':
    print(1)

1
